In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.1'
# spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Hit:1 https://developer.download.nvidia.com/compute

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

--2022-06-22 12:08:21--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.56MB/s    in 0.2s    

2022-06-22 12:08:22 (4.56 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
# Read in outcomes data from cleaned datafiles
from pyspark import SparkFiles
outcomes_url="https://purifai.s3.us-west-1.amazonaws.com/clean-data/cleaned-outcomes.csv"
spark.sparkContext.addFile(outcomes_url)
cleaned_outcomes_df = spark.read.csv(SparkFiles.get("cleaned-outcomes.csv"), sep=",", header=True, inferSchema=False)

# Show outcomes DataFrame
cleaned_outcomes_df.show()

+--------------------+------------------+---------------------+----------+---------------+--------------+-----------+-------------+---------------------+--------------------+------------------+--------------------+--------------+------------------+----------------+-------------------------+-------------------------+-------------+--------------+
|           sample_id|      structure_id|preferred_lcms_method|spe_method|         method|spe_successful|crashed_out|sample_status|sample_current_status|   termination_cause|  termination_step| termination_details|reaction_scale|selected_fractions|volume_collected|total_fractions_collected|recovered_sample_dry_mass|percent_yield|percent_purity|
+--------------------+------------------+---------------------+----------+---------------+--------------+-----------+-------------+---------------------+--------------------+------------------+--------------------+--------------+------------------+----------------+-------------------------+---------------

In [4]:
# Read in structures data from cleaned datafiles
structures_url="https://purifai.s3.us-west-1.amazonaws.com/clean-data/cleaned-structures.csv"
spark.sparkContext.addFile(structures_url)
cleaned_structures_df = spark.read.csv(SparkFiles.get("cleaned-structures.csv"), sep=",", header=True, inferSchema=False)

# Show outcomes DataFrame
cleaned_structures_df.show()

+---------------+-------+-----------+-----+-----------+--------------+-------+-------+------------+-------------------+----------------+----------------+----------------+-----------+-----------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------------+--------------+-----------+-----------+--------------+---------+-------+-----------------------+------------------------+-----------------+----------------------+-----------------------+----------------+-------------+----------+--------------+-----------------+-----------------------+------------------------+-----------------+---------+
|   structure_id|  MolWt| ExactMolWt| TPSA|        qed|HeavyAtomMolWt|MolLogP|  MolMR|FractionCSP3|NumValenceElectrons|MaxPartialCharge|MinPartialCharge|FpDensityMorgan1|   BalabanJ|    BertzCT|HallKierAlpha|        Ipc|     Kappa2|  LabuteASA| PEOE_VSA10|  PEOE_VSA2|  SMR_VSA10|   SMR_VSA4| SlogP_VSA2| SlogP_VSA6|MaxEStateIndex

In [6]:
# Configure settings for RDS
# from './config.py' import db_password
mode = "append"
jdbc_url="jdbc:postgresql://purifai.ceoinb9nwfxg.us-west-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "purifai2022", 
          "driver":"org.postgresql.Driver"}

In [8]:
# Write DataFrame to active_user table in RDS

cleaned_outcomes_df.write.jdbc(url=jdbc_url, table='outcomes', mode=mode, properties=config)
cleaned_structures_df.write.jdbc(url=jdbc_url, table='structures', mode=mode, properties=config)
# SPE_analysis_df.write.jdbc(url=jdbc_url, table='SPE_analysis', mode=mode, properties=config)